In [3]:
import hashlib
import random
from phe import paillier

# ======= 参数设置 =======
P1_ids = ['user1', 'user2', 'user3', 'user4']
P2_data = [('user2', 50), ('user3', 30), ('user5', 90)]
PRIME = 2**521 - 1  # 模拟群 G（使用大素数）

# 哈希到群元素 G
def H(x: str) -> int:
    return int(hashlib.sha256(x.encode()).hexdigest(), 16) % PRIME

# ======= Round 0: 密钥生成 =======
# Paillier 密钥对（P2 生成公钥发给 P1）
pk, sk = paillier.generate_paillier_keypair()

# 双方各自选择随机指数 k1, k2
k1 = random.randint(2, PRIME-1)
k2 = random.randint(2, PRIME-1)

# ======= Round 1: P1 → P2: 发送 {H(vi)^k1} =======
P1_masked = [pow(H(vi), k1, PRIME) for vi in P1_ids]
random.shuffle(P1_masked)

# ======= Round 2: P2 → P1 =======
# Step 1: 对 Round1 的值再进行 ^k2 并发送 Z
Z = [pow(x, k2, PRIME) for x in P1_masked]
Z_set = set(Z)

# Step 2: 计算 {(H(wj)^k2, Enc(tj))}
masked_enc_list = []
for wj, tj in P2_data:
    h = H(wj)
    h_k2 = pow(h, k2, PRIME)
    enc_tj = pk.encrypt(tj)
    masked_enc_list.append((h_k2, enc_tj))
random.shuffle(masked_enc_list)

# ======= Round 3: P1 处理交集与加和 =======
intersection_sum = pk.encrypt(0)
hits = []

for h_k2, enc in masked_enc_list:
    h_k1k2 = pow(h_k2, k1, PRIME)
    if h_k1k2 in Z_set:
        intersection_sum += enc
        hits.append(h_k1k2)

# ======= Round 4: P1 → P2 解密 =======
final_sum = sk.decrypt(intersection_sum)

# ======= 输出结果 =======
print(f"✅ 交集总金额：{final_sum}")
print(f"🔍 命中项（H^k1k2）前缀：{[str(h)[:16] for h in hits]}")

expected = set([uid for uid, _ in P2_data]) & set(P1_ids)
print(f"🧠 真实交集用户: {expected}")



✅ 交集总金额：80
🔍 命中项（H^k1k2）前缀：['3438733556077085', '9033213999627467']
🧠 真实交集用户: {'user2', 'user3'}
